# L3 — Chains and Runnables (LangChain v1) — **Single End‑to‑End Chain**

This rewritten notebook builds **one composed LCEL chain** that runs the whole workflow:

**topic → outline → expanded paragraph → summary**

It also shows *exactly* how values are passed between steps using LCEL and `RunnablePassthrough.assign(...)`.

> You can call the final chain with either a **string** (`"my topic"`) or a **dict** (`{"topic": "my topic"}`).


## 1) Setup

Make sure you have an OpenAI API key in your environment:

- `OPENAI_API_KEY`

Then restart the kernel so the environment variable is picked up.


In [5]:
import os

assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment and restart the kernel."

MODEL = "gpt-5-mini"  # change if you want
print("Using model:", MODEL)


Using model: gpt-5-mini


## 2) Imports

Key concepts used below:

- **LCEL**: the `|` operator builds a runnable pipeline.
- **RunnableLambda**: wraps a Python function as a runnable.
- **RunnablePassthrough.assign(...)**: adds new keys to a dict while keeping existing keys.
- **StrOutputParser**: converts the LLM response into a Python `str`.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


## 3) Create the model and output parser

In [6]:
llm = ChatOpenAI(model=MODEL)
to_text = StrOutputParser()


## 4) Prompts

Each prompt below expects a specific **input key**:

- Outline prompt expects: `{"topic": ...}`
- Expand prompt expects: `{"outline": ...}`
- Summary prompt expects: `{"text": ...}`


In [7]:
outline_prompt = ChatPromptTemplate.from_messages([
    ("system", "Create a compact outline with 4 bullet points."),
    ("user", "{topic}")
])

expand_prompt = ChatPromptTemplate.from_messages([
    ("system", "Expand bullet point #2 into a short paragraph."),
    ("user", "Outline:\n{outline}")
])

summ_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize the text in 2 sentences."),
    ("user", "{text}")
])


## 5) Build small chains

These are the *atomic* pieces:

- `outline_chain`: dict with `topic` → outline string  
- `expand_chain`: dict with `outline` → expanded paragraph string  
- `summ_chain`: dict with `text` → summary string


In [8]:
outline_chain = outline_prompt | llm | to_text
expand_chain = expand_prompt | llm | to_text
summ_chain = summ_prompt | llm | to_text


## 6) Compose one end‑to‑end chain (LCEL)

### The pattern

We keep the input as a **dict** and progressively *add* new keys:

1. Ensure input is a dict with key `topic`.
2. Add `outline` by running `outline_chain` on the dict.
3. Add `expanded` by running `expand_chain` on a dict containing only the `outline`.
4. Add `summary` by running `summ_chain` on a dict containing only the `text`.

### Why it’s less confusing

- Every step has a clear input/output shape.
- `assign(...)` preserves the running “state” dict so you can see everything at the end.


In [9]:
# Accept either:
# - a raw string topic: "my topic"
# - or a dict: {"topic": "my topic"}
normalize_input = RunnableLambda(
    lambda x: {"topic": x} if isinstance(x, str) else x
)

# Helper runnables to reshape the dict for downstream chains
to_outline_input = RunnableLambda(lambda d: {"outline": d["outline"]})
to_summary_input = RunnableLambda(lambda d: {"text": d["expanded"]})

end_to_end_chain = (
    normalize_input
    # Add outline while keeping {"topic": ...}
    | RunnablePassthrough.assign(outline=outline_chain)
    # Add expanded paragraph (expand_chain expects {"outline": ...})
    | RunnablePassthrough.assign(expanded=(to_outline_input | expand_chain))
    # Add summary (summ_chain expects {"text": ...})
    | RunnablePassthrough.assign(summary=(to_summary_input | summ_chain))
)

end_to_end_chain


RunnableLambda(lambda x: {'topic': x} if isinstance(x, str) else x)
| RunnableAssign(mapper={
    outline: ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Create a compact outline with 4 bullet points.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic}'), additional_kwargs={})])
             | ChatOpenAI(profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resou

## 7) Run it

The output is a single dict containing all intermediate artifacts:

- `topic`
- `outline`
- `expanded`
- `summary`


In [10]:
topic = "How to write reliable prompts for LLMs"

result = end_to_end_chain.invoke(topic)  # you can also pass {"topic": topic}

print("=== TOPIC ===")
print(result["topic"])
print("\n=== OUTLINE ===")
print(result["outline"])
print("\n=== EXPANDED (bullet #2) ===")
print(result["expanded"])
print("\n=== SUMMARY ===")
print(result["summary"])


=== TOPIC ===
How to write reliable prompts for LLMs

=== OUTLINE ===
- State the goal and success criteria up front: name the task, the model’s role (e.g., "act as a technical reviewer"), the required audience, and the exact output format you want (JSON, bullet list, code, length).
- Give essential context and examples: include all facts the model needs and 1–3 few‑shot examples or templates so it learns the desired structure and content.
- Add explicit constraints and style rules: specify tone, level of detail, forbidden content, stepwise reasoning if needed (e.g., "show steps" vs "only final answer"), and any formatting rules.
- Iterate and validate: test with varied inputs, run adversarial/edge cases, refine prompts that produce errors, and use temperature/beam settings or follow‑up verification prompts to improve reliability.

=== EXPANDED (bullet #2) ===
Give the model all essential context and concrete examples so it knows what to produce: include the relevant facts, background,

## 8) (Optional) Inspect what flows through the chain

If you’re learning LCEL, it can help to print the “state dict” after each stage.
Below we build the same chain in 3 visible steps so you can see the shapes.


In [11]:
state0 = normalize_input.invoke(topic)
print("state0:", state0)

state1 = (normalize_input | RunnablePassthrough.assign(outline=outline_chain)).invoke(topic)
print("\nstate1 keys:", state1.keys())

state2 = (
    normalize_input
    | RunnablePassthrough.assign(outline=outline_chain)
    | RunnablePassthrough.assign(expanded=(to_outline_input | expand_chain))
).invoke(topic)
print("\nstate2 keys:", state2.keys())

state3 = end_to_end_chain.invoke(topic)
print("\nstate3 keys:", state3.keys())


state0: {'topic': 'How to write reliable prompts for LLMs'}

state1 keys: dict_keys(['topic', 'outline'])

state2 keys: dict_keys(['topic', 'outline', 'expanded'])

state3 keys: dict_keys(['topic', 'outline', 'expanded', 'summary'])


### Same Logic Without LCL

In [14]:
topic = "How to write reliable prompts for LLMs"

def end_to_end_plain(raw_input):
    # normalize_input
    if isinstance(raw_input, str):
        state = {"topic": raw_input}
    else:
        state = raw_input

    # outline
    state["outline"] = outline_chain.invoke(state)

    # expanded
    expand_input = {"outline": state["outline"]}
    state["expanded"] = expand_chain.invoke(expand_input)

    # summary
    summary_input = {"text": state["expanded"]}
    state["summary"] = summ_chain.invoke(summary_input)

    return state

result = end_to_end_plain(topic)

print("=== TOPIC ===")
print(result["topic"])
print("\n=== OUTLINE ===")
print(result["outline"])
print("\n=== EXPANDED (bullet #2) ===")
print(result["expanded"])
print("\n=== SUMMARY ===")
print(result["summary"])



=== TOPIC ===
How to write reliable prompts for LLMs

=== OUTLINE ===
- State the goal, role, audience, and explicit success criteria (what the output must accomplish).
- Give necessary context and concrete examples or input-output pairs so the model knows desired content and tone.
- Specify format, length, style, constraints, and any stepwise process (e.g., "list, 5 items, concise, include sources").
- Test with edge cases, measure output quality, iterate prompts, and add guardrails (clarifications, rejection rules, sampling/temperature settings).

=== EXPANDED (bullet #2) ===
Provide the model with all necessary background and plenty of concrete examples so it clearly understands the desired content and tone: explain relevant domain facts, the audience’s prior knowledge, any constraints or assumptions, and then show sample inputs with ideal outputs (and, if helpful, counter-examples of what to avoid). Include examples that illustrate structure, phrasing, level of detail, and voice (f